In [1]:
import findspark
findspark.init("/home/ubuntu/spark-2.1.1-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BDAS-Linda').getOrCreate()

from functools import reduce
from pyspark.sql import DataFrame


In [2]:
# import Maori language in education datasets
filename1 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2004-2008.csv"
filename2 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2009-2013.csv"
filename3 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2014-2018.csv"
filename4 = "./Maori-Language-Learning-Student-Numbers-by-Ethnicity-2019.csv"
MLLSN1=spark.read.load(filename1, format="csv", header="true")
MLLSN2=spark.read.load(filename2, format="csv", header="true")
MLLSN3=spark.read.load(filename3, format="csv", header="true")
MLLSN4=spark.read.load(filename4, format="csv", header="true")

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

MLLSN=unionAll(MLLSN1,MLLSN2,MLLSN3,MLLSN4)
MLLSN.columns

['Students Number',
 'Year: As at 1 July',
 'Student: Ethnicity',
 'Student: Year level',
 'Student: Year level (Grouped)',
 'Student: Maori Language Immersion Level',
 'Student: Maori Language in Education level',
 'School: ID',
 'School: Maori Language Descriptor',
 'School: Medium',
 'School: Highest Maori Language Immersion Level',
 'School: Name',
 'School: Sector',
 'School: Type',
 'School: Authority',
 'School: Decile',
 'School: Gender',
 'School: Affiliation Type',
 'School: Affiliation',
 'School: Definition',
 'School: Kura Type',
 'Region: Education Region',
 'Region: Regional Council',
 'Region: Territorial Authority',
 'Region: TA with Auckland wards',
 'Region: TA with Auckland local boards',
 'Region: MOE Local office',
 'Region: Census area unit',
 'Region: Ward']

In [3]:
# import population data
filename = "./Subnational-ethnic-population-projections-2013base.csv"
POP=spark.read.load(filename, format="csv", header="true")
POP.columns

['TA with Auckland Local Board',
 'Year at 30 June',
 'Euro-all-ages',
 'Euro-5-9',
 'Euro-10-14',
 'Euro-15-19',
 'Maori-all-ages',
 'Maori-5-9',
 'Maori-10-14',
 'Maori-15-19',
 'Asian-all-ages',
 'Asian-5-9',
 'Asian-10-14',
 'Asian-15-19',
 'Pacific-all-ages',
 'Pacific-5-9',
 'Pacific-10-14',
 'Pacific-15-19',
 'Total-all-ages',
 'Total-5-9',
 'Total-10-14',
 'Total-15-19']

In [4]:
MLLSN.count()

624768

In [5]:
MLLSN.printSchema()

root
 |-- Students Number: string (nullable = true)
 |-- Year: As at 1 July: string (nullable = true)
 |-- Student: Ethnicity: string (nullable = true)
 |-- Student: Year level: string (nullable = true)
 |-- Student: Year level (Grouped): string (nullable = true)
 |-- Student: Maori Language Immersion Level: string (nullable = true)
 |-- Student: Maori Language in Education level: string (nullable = true)
 |-- School: ID: string (nullable = true)
 |-- School: Maori Language Descriptor: string (nullable = true)
 |-- School: Medium: string (nullable = true)
 |-- School: Highest Maori Language Immersion Level: string (nullable = true)
 |-- School: Name: string (nullable = true)
 |-- School: Sector: string (nullable = true)
 |-- School: Type: string (nullable = true)
 |-- School: Authority: string (nullable = true)
 |-- School: Decile: string (nullable = true)
 |-- School: Gender: string (nullable = true)
 |-- School: Affiliation Type: string (nullable = true)
 |-- School: Affiliation: str

In [6]:
POP.count()

528

In [7]:
POP.printSchema()

root
 |-- TA with Auckland Local Board: string (nullable = true)
 |-- Year at 30 June: string (nullable = true)
 |-- Euro-all-ages: string (nullable = true)
 |-- Euro-5-9: string (nullable = true)
 |-- Euro-10-14: string (nullable = true)
 |-- Euro-15-19: string (nullable = true)
 |-- Maori-all-ages: string (nullable = true)
 |-- Maori-5-9: string (nullable = true)
 |-- Maori-10-14: string (nullable = true)
 |-- Maori-15-19: string (nullable = true)
 |-- Asian-all-ages: string (nullable = true)
 |-- Asian-5-9: string (nullable = true)
 |-- Asian-10-14: string (nullable = true)
 |-- Asian-15-19: string (nullable = true)
 |-- Pacific-all-ages: string (nullable = true)
 |-- Pacific-5-9: string (nullable = true)
 |-- Pacific-10-14: string (nullable = true)
 |-- Pacific-15-19: string (nullable = true)
 |-- Total-all-ages: string (nullable = true)
 |-- Total-5-9: string (nullable = true)
 |-- Total-10-14: string (nullable = true)
 |-- Total-15-19: string (nullable = true)



In [8]:
MLLSN.show()

+---------------+------------------+------------------+-------------------+-----------------------------+---------------------------------------+------------------------------------------+----------+---------------------------------+--------------------+----------------------------------------------+--------------------+--------------+--------------------+--------------------+--------------+--------------+------------------------+-------------------+------------------+-----------------+------------------------+------------------------+-----------------------------+------------------------------+-------------------------------------+------------------------+------------------------+--------------------+
|Students Number|Year: As at 1 July|Student: Ethnicity|Student: Year level|Student: Year level (Grouped)|Student: Maori Language Immersion Level|Student: Maori Language in Education level|School: ID|School: Maori Language Descriptor|      School: Medium|School: Highest Maori Language Imm

In [21]:
#check unique values in the key attributes
MLLSN.select("Student: Ethnicity").distinct().show()
MLLSN.select("Student: Year level (Grouped)").distinct().show()

+------------------+
|Student: Ethnicity|
+------------------+
|             Maori|
|         Non Maori|
+------------------+

+-----------------------------+
|Student: Year level (Grouped)|
+-----------------------------+
|          Secondary (Year 9+)|
|           Primary (Year 1-8)|
+-----------------------------+



In [23]:
MLLSN.select("Student: Maori Language Immersion Level").distinct().show()
MLLSN.select("Student: Maori Language in Education level").distinct().show()

+---------------------------------------+
|Student: Maori Language Immersion Level|
+---------------------------------------+
|                   Level 4(b): At le...|
|                   Level 4(a): up to...|
|                   Level 5: Less tha...|
|                        Level 3: 31-50%|
|                   No Maori language...|
|                       Level 1: 81-100%|
|                    Level 6: Taha Maori|
|                        Level 2: 51-80%|
+---------------------------------------+

+------------------------------------------+
|Student: Maori Language in Education level|
+------------------------------------------+
|                      Maori Language in...|
|                      No Maori language...|
|                              Maori medium|
+------------------------------------------+



In [24]:
MLLSN.select('School: Type').distinct().show()
MLLSN.select('School: Sector').distinct().show()

+--------------------+
|        School: Type|
+--------------------+
|Kura Teina - Primary|
|Restricted Compos...|
|      Special School|
|Correspondence Sc...|
|        Full Primary|
|        Intermediate|
|    Teen Parent Unit|
|Secondary (Year 9...|
|Kura Teina - Comp...|
|        Contributing|
|Secondary (Year 7...|
|Composite (Year 1...|
+--------------------+

+--------------+
|School: Sector|
+--------------+
|     Secondary|
|       Special|
|     Composite|
|       Primary|
+--------------+



In [10]:
POP.show()

+----------------------------+---------------+-------------+--------+----------+----------+--------------+---------+-----------+-----------+--------------+---------+-----------+-----------+----------------+-----------+-------------+-------------+--------------+---------+-----------+-----------+
|TA with Auckland Local Board|Year at 30 June|Euro-all-ages|Euro-5-9|Euro-10-14|Euro-15-19|Maori-all-ages|Maori-5-9|Maori-10-14|Maori-15-19|Asian-all-ages|Asian-5-9|Asian-10-14|Asian-15-19|Pacific-all-ages|Pacific-5-9|Pacific-10-14|Pacific-15-19|Total-all-ages|Total-5-9|Total-10-14|Total-15-19|
+----------------------------+---------------+-------------+--------+----------+----------+--------------+---------+-----------+-----------+--------------+---------+-----------+-----------+----------------+-----------+-------------+-------------+--------------+---------+-----------+-----------+
|          Far North district|           2013|        40800|    2750|      2770|      2420|         27100|     2